In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

# Import Pyod and the methods

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

c:\python3764\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Import Metrics Package

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [4]:
from glob import glob, iglob
mat_file_list = [i for i in glob('.\mat_files\*.mat')]
mat_file_list

['.\\mat_files\\arrhythmia.mat',
 '.\\mat_files\\cardio.mat',
 '.\\mat_files\\glass.mat',
 '.\\mat_files\\ionosphere.mat',
 '.\\mat_files\\letter.mat',
 '.\\mat_files\\lympho.mat',
 '.\\mat_files\\mnist.mat',
 '.\\mat_files\\musk.mat',
 '.\\mat_files\\optdigits.mat',
 '.\\mat_files\\pendigits.mat',
 '.\\mat_files\\pima.mat',
 '.\\mat_files\\satellite.mat',
 '.\\mat_files\\satimage-2.mat',
 '.\\mat_files\\shuttle.mat',
 '.\\mat_files\\vertebral.mat',
 '.\\mat_files\\vowels.mat',
 '.\\mat_files\\wbc.mat']

# Specifying the columns

In [5]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

# Precision Time and Roc evolution tables creation 

In [6]:
roc_df = prn_df = time_df = pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


# Exploring Mat files

In [7]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
        print(f'\n... Processing {mat_file}...')
        mat = loadmat(mat_file)

        X = mat['X']
        y = mat['y'].ravel()
        outliers_fraction = np.count_nonzero(y) / len(y)
        outliers_percentage = round(outliers_fraction * 100, ndigits=4)

        # construct containers for saving results
        roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
        prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
        time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

        # 60% data for training and 40% for testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = random_state)

        # standardizing data for processing
        X_train_norm, X_test_norm = standardizer(X_train, X_test)
        
        classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
                       
                       'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction, check_estimator=False,
                                                               random_state=random_state),
                       
                       'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                       
                       'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                       
                       'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
                       
                       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                       
                       'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction),
                       
                       'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                       
                       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                       
                       'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)
                      }
        
        for clf_name, clf in classifiers.items():
            t0 = time()
            
            clf.fit(X_train_norm)
            
            test_scores = clf.decision_function(X_test_norm)
            t1 = time()
            
            duration = round(t1 - t0, ndigits=4)
            time_list.append(duration)
            
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
            
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
            
            print(f'{clf_name} \n\tROC: {roc} \tprecision @ rank n: {prn} \texecution time: {duration}s')
            
            roc_list.append(roc)
            prn_list.append(prn)
            
        temp_df = pd.DataFrame(time_list).transpose()
        temp_df.columns = df_columns
        time_df = pd.concat([time_df, temp_df], axis=0)
        
        temp_df = pd.DataFrame(roc_list).transpose()
        temp_df.columns = df_columns
        roc_df = pd.concat([roc_df, temp_df], axis=0)
        
        temp_df = pd.DataFrame(prn_list).transpose()
        temp_df.columns = df_columns
        prn_df = pd.concat([prn_df, temp_df], axis=0)
        
        print('\n****************************************')


... Processing .\mat_files\arrhythmia.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.7687 	precision @ rank n: 0.3571 	execution time: 6.2814s


c:\python3764\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Cluster-based Local Outlier Factor 
	ROC: 0.7684 	precision @ rank n: 0.4643 	execution time: 6.4647s
Feature Bagging 
	ROC: 0.7799 	precision @ rank n: 0.5 	execution time: 2.1499s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.8511 	precision @ rank n: 0.5714 	execution time: 8.0248s
Isolation Forest 
	ROC: 0.8478 	precision @ rank n: 0.5357 	execution time: 1.555s
K Nearest Neighbors (KNN) 
	ROC: 0.782 	precision @ rank n: 0.5 	execution time: 0.366s
Local Outlier Factor (LOF) 
	ROC: 0.7787 	precision @ rank n: 0.4643 	execution time: 0.2849s


c:\python3764\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) 
	ROC: 0.8228 	precision @ rank n: 0.4286 	execution time: 17.1994s
One-class SVM (OCSVM) 
	ROC: 0.7986 	precision @ rank n: 0.5 	execution time: 0.165s
Principal Component Analysis (PCA) 
	ROC: 0.7997 	precision @ rank n: 0.5 	execution time: 0.5941s

****************************************

... Processing .\mat_files\cardio.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.5763 	precision @ rank n: 0.1875 	execution time: 1.592s


...


Feature Bagging 
	ROC: 0.3027 	precision @ rank n: 0.0 	execution time: 0.1681s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.2695 	precision @ rank n: 0.0 	execution time: 0.01s
Isolation Forest 
	ROC: 0.3576 	precision @ rank n: 0.0 	execution time: 1.066s
K Nearest Neighbors (KNN) 
	ROC: 0.318 	precision @ rank n: 0.0 	execution time: 0.045s
Local Outlier Factor (LOF) 
	ROC: 0.318 	precision @ rank n: 0.0 	execution time: 0.016s
Minimum Covariance Determinant (MCD) 
	ROC: 0.3308 	precision @ rank n: 0.0 	execution time: 0.2699s
One-class SVM (OCSVM) 
	ROC: 0.4087 	precision @ rank n: 0.0 	execution time: 0.0081s
Principal Component Analysis (PCA) 
	ROC: 0.3397 	precision @ rank n: 0.0 	execution time: 0.008s

****************************************

... Processing .\mat_files\vowels.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.9521 	precision @ rank n: 0.4706 	execution time: 1.286s


c:\python3764\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Cluster-based Local Outlier Factor 
	ROC: 0.9278 	precision @ rank n: 0.4118 	execution time: 0.419s
Feature Bagging 
	ROC: 0.9385 	precision @ rank n: 0.3529 	execution time: 1.135s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.6758 	precision @ rank n: 0.1765 	execution time: 0.0181s
Isolation Forest 
	ROC: 0.7469 	precision @ rank n: 0.1176 	execution time: 1.416s
K Nearest Neighbors (KNN) 
	ROC: 0.9568 	precision @ rank n: 0.5294 	execution time: 0.381s
Local Outlier Factor (LOF) 
	ROC: 0.9345 	precision @ rank n: 0.4118 	execution time: 0.127s
Minimum Covariance Determinant (MCD) 
	ROC: 0.6779 	precision @ rank n: 0.0 	execution time: 5.1985s
One-class SVM (OCSVM) 
	ROC: 0.7415 	precision @ rank n: 0.2941 	execution time: 0.125s
Principal Component Analysis (PCA) 
	ROC: 0.5787 	precision @ rank n: 0.1176 	execution time: 0.015s

****************************************

... Processing .\mat_files\wbc.mat...
Angle-based Outlier Detector (ABOD) 
	ROC: 0.9232 	precision @ rank n: 

c:\python3764\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Cluster-based Local Outlier Factor 
	ROC: 0.9063 	precision @ rank n: 0.6 	execution time: 0.227s
Feature Bagging 
	ROC: 0.9415 	precision @ rank n: 0.5 	execution time: 0.331s
Histogram-base Outlier Detection (HBOS) 
	ROC: 0.9592 	precision @ rank n: 0.7 	execution time: 0.04s
Isolation Forest 
	ROC: 0.9451 	precision @ rank n: 0.5 	execution time: 1.127s
K Nearest Neighbors (KNN) 
	ROC: 0.9437 	precision @ rank n: 0.5 	execution time: 0.1029s
Local Outlier Factor (LOF) 
	ROC: 0.9352 	precision @ rank n: 0.4 	execution time: 0.04s
Minimum Covariance Determinant (MCD) 
	ROC: 0.8986 	precision @ rank n: 0.4 	execution time: 1.5607s
One-class SVM (OCSVM) 
	ROC: 0.9408 	precision @ rank n: 0.5 	execution time: 0.022s
Principal Component Analysis (PCA) 
	ROC: 0.9324 	precision @ rank n: 0.6 	execution time: 0.011s

****************************************


In [8]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\mat_files\arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,.\mat_files\cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,.\mat_files\glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,.\mat_files\ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,.\mat_files\letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,.\mat_files\lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9127,0.9636,0.9818
0,.\mat_files\mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7813,0.8409,0.7085,0.863,0.8417,0.8396
0,.\mat_files\musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9992,0.7348,0.5323,1,1,1
0,.\mat_files\optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5442,0.3824,0.4584,0.3486,0.4972,0.504
0,.\mat_files\pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9482,0.7602,0.481,0.8271,0.93,0.9332


In [9]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\mat_files\arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,.\mat_files\cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,.\mat_files\glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,.\mat_files\ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,.\mat_files\letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,.\mat_files\lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,.\mat_files\mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.3116,0.4144,0.339,0.3973,0.3801,0.3767
0,.\mat_files\musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9,0.2333,0.1333,0.9667,1,1
0,.\mat_files\optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,.\mat_files\pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2615,0.0462,0.0462,0.0615,0.2923,0.3385


In [10]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,.\mat_files\arrhythmia,452,274,14.6018,6.2814,6.4647,2.1499,8.0248,1.555,0.366,0.2849,17.1994,0.165,0.5941
0,.\mat_files\cardio,1831,21,9.6122,1.592,0.5301,2.837,0.0311,1.4999,0.676,0.3921,7.3531,0.27,0.031
0,.\mat_files\glass,214,9,4.2056,0.166,0.1531,0.162,0.013,1.24,0.05,0.015,0.297,0.007,0.007
0,.\mat_files\ionosphere,351,33,35.8974,0.287,0.202,0.348,0.04,1.142,0.094,0.031,1.59,0.0211,0.012
0,.\mat_files\letter,1600,32,6.25,1.785,0.433,2.788,0.038,1.469,0.646,0.379,25.2193,0.2809,0.06
0,.\mat_files\lympho,148,18,4.0541,0.109,0.176,0.172,0.022,1.0839,0.027,0.01,0.5114,0.007,0.009
0,.\mat_files\mnist,7603,100,9.2069,26.436,2.184,196.535,0.1971,7.539,24.372,23.256,71.2389,16.922,0.671
0,.\mat_files\musk,3062,166,3.1679,8.28,0.9399,53.759,0.265,5.081,7.381,6.5229,317.72,4.5499,1.0375
0,.\mat_files\optdigits,5216,64,2.8758,10.4031,1.351,58.81,0.129,3.5737,7.142,6.431,31.7476,5.182,0.312
0,.\mat_files\pendigits,6870,16,2.2707,5.873,1.023,17.92,0.0359,2.587,2.853,2.341,17.7807,3.475,0.047
